In [8]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [9]:
# The goal of this file is to show that for large datasets:
#  1) we can solve fair PCA or multi-criteria dimensionality reduction quickly.
# the objectives used are 1) variance 2) mar-loss (both are similar - just a constant shift), and 3) NSW
# The NSW does not fit into MW framework. However, the problem has quick linear oracle, namely SVD, so we will use Frank-Wolfe for NSW.

# 2) The heuristics have loose theory bound, but we can tune more aggressively and certify solutions by duality gap

from optimizer import fairDimReduction_MW, FW
from utils import getObj

We perform three experiments MM_loss, MM_var, and NSW on dimension n=1000. Note that the code below requires that you have preprocess the matrix data of shape 1000 x 1000 in folder 'mat'.

In [10]:
B=[]
m=661967 #total number of datapoints
n=1000
k=16
group_sizes = np.load('group_sizes.npy')
for idx in range(k):
    #load data, use standard normalization on group size by equalizing importance of each group
    B.append(np.load('./mat/n='+str(n)+'_group'+str(idx+1)+'.npy')/group_sizes[idx]*1000)
    # times 1000 so the number is not too small for large normalization constant in large dataset

In [12]:
#experiment n=1000; Mar_loss by MW
d=3
#creating adaptive eta
eta = [4 for i in range(10)]
for j in range(100):
    eta = eta + [max(eta[-1]*0.8,2) for i in range(10)]
#do the MW
[Xs,Xavg,stats] = fairDimReduction_MW(n,k,d,B,eta=2.5 #tuned to be 2.7 for static eta
                                      , T=400, verbose = False,Obj='MM_Loss',calculate_dual=True, n_X_last=24,stopping_gap=1.4e-4)
#Xs are several last iterates 
#Xavg is the average iterate
stats


#the run time shows that MW takes 118 iterations and about 0.6 sec/iteration for n=1000,k=16.
# this one has an exact solution from the last iterate. 
# we can also do fixed eta=2.5, gap=1e-4, it stops at T=142 iterates

MW terminated at T= 118  iterations: current iterate solution achieved primal-dual gap of 0.00014
fairDimReduction_MW is called. Total time used is:  81.78476990000001  seconds.
The best solution found from avg and single iterate acheieves primal -0.14150248285517664 . The dual is -0.141363164452153 . Gap is 0.00013931840302364162 which is 0.09855354014149602 %.


,iteration,"minimum of m objective, that iterate","minimum of m objective, avg iterate",dual objective
0,0,-0.302987,-0.302987,-0.093048
1,1,-0.216194,-0.259591,-0.104292
2,2,-0.190379,-0.226033,-0.111062
3,3,-0.160166,-0.205193,-0.114703
4,4,-0.160357,-0.192186,-0.117092
5,5,-0.160229,-0.183104,-0.119297
6,6,-0.159881,-0.176267,-0.121337
7,7,-0.159384,-0.170885,-0.123230
8,8,-0.158823,-0.166594,-0.124987
9,9,-0.158186,-0.163150,-0.126619


In [7]:
#for n=1000, MM_loss, the last iterate (and not the average) is the optimal solution

#since it is the last iterate, it also has exact rank
sorted(np.linalg.eigh(Xs)[0],reverse=True)

[1.0000000000000013,
 0.9999999999999998,
 0.9999999999999979,
 6.526419219871455e-16,
 5.914390572201431e-16,
 4.3007829475639817e-16,
 2.630167598149304e-16,
 2.4946595212598515e-16,
 1.7413945350402935e-16,
 1.5872182617648402e-16,
 1.0780872101241593e-16,
 1.0735282810062058e-16,
 8.310128317517809e-17,
 6.004432239744871e-17,
 3.6174497754473094e-17,
 3.204867335153027e-17,
 2.7737845806861717e-17,
 2.5457763527931308e-17,
 2.4664544704282613e-17,
 2.236572732263492e-17,
 2.2127459841053205e-17,
 2.0051460050179386e-17,
 1.8622219564388308e-17,
 1.7822312480252305e-17,
 1.688405863469376e-17,
 1.5467568369331163e-17,
 1.4369760673093117e-17,
 1.4069538835655974e-17,
 1.2299232158664e-17,
 1.2136325286646103e-17,
 1.1492551509200873e-17,
 1.1440462140009351e-17,
 1.0557961232297456e-17,
 1.0154878044005431e-17,
 9.631612414377909e-18,
 8.893680779613478e-18,
 8.043051192615866e-18,
 7.956038814019482e-18,
 7.5744772395016e-18,
 6.6854885128895525e-18,
 6.316432014786616e-18,
 6.099

In [24]:
#experiment n=1000; MM_var by MW
d=3
#creating changing eta
eta = [20 for i in range(10)]
for j in range(100):
    eta = eta + [max(eta[-1]-0.5,15) for i in range(10)]
[Xs,Xavg,stats] = fairDimReduction_MW(n,k,d,B,eta=15 #15 for tuned static value
                                      , T=400, verbose = False,Obj='MM_Var',calculate_dual=True, n_X_last=24,stopping_gap=5.5e-5)
stats

#the run time shows that MW takes 99 iterations and about 0.6 sec/iteration for n=1000,k=16.
#use eta=15 and it finishes in 458 iterations, 294 seconds. Tuned with changing eta and it was 172 iterations

MW terminated at T= 99  iterations: current iterate solution achieved primal-dual gap of 5.5e-05
fairDimReduction_MW is called. Total time used is:  65.06115780000073  seconds.
The best solution found from avg and single iterate acheieves primal 0.05934995519313093 . The dual is 0.0594046910019137 . Gap is 5.4735808782770123e-05 which is 0.09214054960913243 %.


,iteration,"minimum of m objective, that iterate","minimum of m objective, avg iterate",dual objective
0,0,0.037588,0.037588,0.072520
1,1,0.049790,0.045525,0.063131
2,2,0.051385,0.047479,0.061953
3,3,0.056179,0.049707,0.061201
4,4,0.056511,0.051068,0.060857
5,5,0.057042,0.052097,0.060620
6,6,0.057197,0.052885,0.060440
7,7,0.057205,0.053515,0.060294
8,8,0.057295,0.054031,0.060174
9,9,0.057427,0.054464,0.060072


In [26]:
#experiment n=1000; NSW by FW
d=3
X = FW(n,k,d,B,Obj='NSW',num_iterations=1000,update_rule='1/2')  

#for n=1000,k=16,d=3 FW takes about 0.4 sec/iterations for easy update rule and NSW objective. 

Iterations t=1: primal is -54.45225112073853, dual is 314.605819073732, and gap is 369.05807019447053 which is 1.1730808771467032%. Time taken: 0.7469529000009061 seconds.
Iterations t=2: primal is -47.79907948184656, dual is -30.196155418561233, and gap is 17.60292406328533 which is 0.5829524924376634%. Time taken: 1.5219457000002876 seconds.
Iterations t=3: primal is -45.25262313363766, dual is -39.67688369184895, and gap is 5.575739441788711 which is 0.1405286636191685%. Time taken: 2.276805299999978 seconds.
Iterations t=4: primal is -44.11058253175177, dual is -41.73802274942417, and gap is 2.3725597823276017 which is 0.056844086663408885%. Time taken: 3.005010000000766 seconds.
Iterations t=5: primal is -43.568176615043264, dual is -42.46580817456398, and gap is 1.102368440479283 which is 0.025958965291506556%. Time taken: 3.7398239000012836 seconds.
Iterations t=6: primal is -43.30365121312946, dual is -42.77096920365077, and gap is 0.5326820094786907 which is 0.0124542889580632

Next, we do three experiments MM_loss, MM_var, and NSW on larger dimensions n=2000. Note that the code below requires that you have preprocess the matrix data of shape 2000 x 2000 in folder 'mat'.

In [13]:
C=[]
m=661967 #total number of datapoints
n=2000
k=16
group_sizes = np.load('group_sizes.npy')
for idx in range(k):
    #load data, use standard normalization on group size by equalizing importance of each group
    C.append(np.load('./mat/n='+str(n)+'_group'+str(idx+1)+'.npy')/group_sizes[idx]*1000)
    # times 1000 so the number is not too small for large normalization constant in large dataset

In [28]:
#experiment n=2000; Mar_loss by MW
d=3
[Xs,Xavg,stats] = fairDimReduction_MW(n,k,d,C,eta=3, T=1000, verbose = False,Obj='MM_Loss',calculate_dual=True, n_X_last=24,stopping_gap=1.4e-4)
stats

#the run time shows that MW with eta=3 takes about 3.3 sec/iteration for n=2000,k=16. 5 times longer than n=1000.
#total time about 3000 seconds or 50 minutes.

MW terminated at T= 197  iterations: current iterate solution achieved primal-dual gap of 0.00014
fairDimReduction_MW is called. Total time used is:  588.9505711000002  seconds.
The best solution found from avg and single iterate acheieves primal -0.1482778617047387 . The dual is -0.14813851031809006 . Gap is 0.00013935138664863955 which is 0.09406830563465073 %.


,iteration,"minimum of m objective, that iterate","minimum of m objective, avg iterate",dual objective
0,0,-0.302058,-0.302058,-0.099925
1,1,-0.215791,-0.258925,-0.113424
2,2,-0.183923,-0.220667,-0.120553
3,3,-0.168547,-0.202012,-0.123967
4,4,-0.161478,-0.190985,-0.126880
5,5,-0.161678,-0.183576,-0.129423
6,6,-0.161772,-0.178179,-0.131665
7,7,-0.161715,-0.174045,-0.133649
8,8,-0.161508,-0.170769,-0.135406
9,9,-0.161166,-0.168112,-0.136961


In [29]:
#experiment n=2000; MM_var by MW
d=3
[Xs,Xavg,stats] = fairDimReduction_MW(n,k,d,C,eta=11, T=2000, verbose = False,Obj='MM_Var',calculate_dual=True, n_X_last=24,stopping_gap=5.5e-5)
stats

#the run time shows that MW eith eta=11 takes about 3.3 sec/iteration for n=2000,k=16. 5 times longer than n=1000

MW terminated at T= 213  iterations: current iterate solution achieved primal-dual gap of 5.5e-05
fairDimReduction_MW is called. Total time used is:  585.4562041999998  seconds.
The best solution found from avg and single iterate acheieves primal 0.06155820278322715 . The dual is 0.06161314834004326 . Gap is 5.49455568161078e-05 which is 0.08917829764657215 %.


,iteration,"minimum of m objective, that iterate","minimum of m objective, avg iterate",dual objective
0,0,0.037671,0.037671,0.074670
1,1,0.049961,0.043816,0.066006
2,2,0.053849,0.047160,0.064733
3,3,0.055659,0.049285,0.064078
4,4,0.056906,0.050809,0.063666
5,5,0.057810,0.051976,0.063368
6,6,0.058500,0.052908,0.063134
7,7,0.058987,0.053674,0.062943
8,8,0.059176,0.054318,0.062783
9,9,0.059258,0.054868,0.062646


In [11]:
#experiment n=2000; NSW by FW
d=3
X = FW(n,k,d,C,Obj='NSW',num_iterations=2000,update_rule='1/2_plus_every_10')   #update_rule='1/2_plus_every_10' is the best

#for n=2000,k=16,d=3 FW takes about 2.4 sec/iterations. 6 times longer than n=1000. Total time is 68 seconds

Iterations t=1: gap is 567.3989177873624. Time taken: 2.6074320999999827 seconds.
Iterations t=2: gap is 17.267923541809644. Time taken: 5.901472199999944 seconds.
Iterations t=3: gap is 5.642016448902536. Time taken: 8.394656299999951 seconds.
Iterations t=4: gap is 2.3641087281162476. Time taken: 10.86945490000005 seconds.
Iterations t=5: gap is 1.107104589234063. Time taken: 13.359743200000025 seconds.
Iterations t=6: gap is 0.542752014245574. Time taken: 15.861350900000048 seconds.
Iterations t=7: gap is 0.2757225305399501. Time taken: 18.194097400000032 seconds.
Iterations t=8: gap is 0.14426842739690537. Time taken: 20.885123600000043 seconds.
Iterations t=9: gap is 0.08021346814868753. Time taken: 23.450990000000047 seconds.
Iterations t=10: gap is 0.04715510532777105. Time taken: 26.293430400000034 seconds.
Iterations t=11: gap is 0.031632446048617736. Time taken: 28.82434490000003 seconds.
Iterations t=12: gap is 0.017081301750489226. Time taken: 31.446317099999987 seconds.
It

In [30]:
sorted(np.linalg.eigh(X)[0],reverse=True) #exact rank solution found

[0.9999980475700205,
 0.999997804338386,
 0.9999957104732975,
 2.314036758849733e-06,
 2.773824872395951e-07,
 1.0055119650686998e-07,
 5.230593163284879e-08,
 1.0153343860720396e-08,
 9.070423059666659e-09,
 8.233812473662934e-09,
 6.3837232542658405e-09,
 5.940131260358521e-09,
 5.8215815388611955e-09,
 5.769709455136373e-09,
 5.738986692854279e-09,
 5.734513508623118e-09,
 5.723748645347344e-09,
 5.722365553202477e-09,
 5.722273189010097e-09,
 5.722244296532333e-09,
 5.7221134447412585e-09,
 5.722060359999378e-09,
 5.722046653832653e-09,
 5.722046128148603e-09,
 5.722046105084036e-09,
 5.7220460740661545e-09,
 5.722046065191774e-09,
 5.722046044966854e-09,
 5.7220460435800284e-09,
 5.722046010913335e-09,
 5.722045989022031e-09,
 5.722045971368563e-09,
 5.7220459643916775e-09,
 5.722045962979465e-09,
 5.722045951603119e-09,
 5.722045948019013e-09,
 5.722045942763104e-09,
 5.722045940133739e-09,
 5.722045932166854e-09,
 5.72204593108737e-09,
 5.7220459300249055e-09,
 5.722045929099166